Repovizz2 Tutorial
==================

* Upload
* Retrieve
* Edit

Repovizz2 DataPack Upload Example
=================================

This part demonstrates basic datapack operations (upload)

We will use here the repovizz2 client library to ease the authentication, but it is basically a standard OAuth2 authentication.

In [ ]:
from repovizz2 import RepoVizzClient

We will be using those modules:

In [ ]:
import json
# This is for the notebook
from IPython.display import IFrame, display

Now, some global parameters to connect to repovizz2

In [ ]:
CLIENT_ID = "1d77f100-dab1-4e99-84ae-3bf36584aafc"
CLIENT_SECRET = "031a141c-65fe-4143-9884-e637128ccdbc"


Repovizz2 authentication
-----------------------

We authenticate the client (this script) against repoVizz2 using a registered user. For more details on the authentication system see [repoVizz2 authentication](Authentication.ipynb).


This will create an iframe in the notebook to authenticate the client, but you can open a new browser to do so, if you want

In [ ]:
repovizz_client = RepoVizzClient(client_id=CLIENT_ID,client_secret=CLIENT_SECRET)
authorization_url = repovizz_client.start_auth()
repovizz_client.start_server(async=True)

# print authorization_url # and click on it
# - or -
# import webbrowser
# webbrowser.open(authorization_url)
# - or -
display(IFrame(authorization_url,800,600))

In [ ]:
repovizz_client.finish_auth()

Retrieving identity data
------------------------

We first get info about our identity. We can use `/api/v1.0/user` to get json information about the current logged user.

In [ ]:
myself = repovizz_client.get("/api/v1.0/user")
username = myself['username']
userid = myself['id']
print myself

Uploading a Datapack
--------------------

To upload a datapack:

* Upload the Datapack structure
* Upload its data files

Uploading the structure
-----------------------

Now we upload the datapack (without the data). The result of the call will include the upload datapack (item), including its ID. To do so, we post to `/api/v1.0/datapacks` with a new datapack using json.

In [ ]:
datapack_json = json.load(open('MYO_example_datapack/MYO_example_datapack.json'))
result = repovizz_client.post(
    "/api/v1.0/datapacks",
    json={
        'structure': datapack_json,
        'name': datapack_json['info']['name'],
        'owner': userid,
    },
)
datapack = result['item']

Listing the data files to upload
--------------------------------

Datapacks usually refer to external material, such as `csv` files, videos or audio. Those were'nt uploaded to the server yet. Instead, reference to their filenames were used as handles.

To upload all the real assets, first we can ask for all the data files referenced in the datapack using `/api/v1.0/datapacks/{datapack_id}/content`. This returns us a list of the referenced filenames.

In [ ]:
datapack_id = datapack['id']
data_in_datapack = repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()
data_in_datapack

Uploading the data files
------------------------

For each of those files, we have to send the actual content to be saved in the server. We can do that by posting to `/api/v1.0/datapacks/{datapack_id}/content/{content_id}` with the actual file attached.

Repovizz2 will substitute the content identifier with a new one, using the format `repovizz:{hashcode}`, meaning that the contents of this file can be found in the repovizz2 server, with the _hashcode_ as its handle.

In [ ]:
for fname in data_in_datapack:
    print repovizz_client.post(
        '/api/v1.0/datapacks/{}/content/{}'.format(datapack_id,fname),
        files={fname:open('MYO_example_datapack/'+fname)}
    )


Checking that everything has been properly uploaded
---------------------------------------------------

We can now check that indeed we changed the references to the contents in the datapack by quering again `/api/v1.0/datapacks/{datapack_id}/content`.

In [ ]:
data_in_datapack = repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()
data_in_datapack

With this step completed, we concluded uploading the datapack to repovizz2. See [Retrieval](Retrieval.ipynb) for the next steps.

Retrieve Datapack's data
========================

Repovizz2 can be used to retrieve datapacks with multimodal data, previously uploaded by yourself or other users.


Listing my datapacks
--------------------

In [7]:
myself = repovizz_client.get("/api/v1.0/user")
print myself['datapacks']

[u'a4e6b685-afb8-4b40-9c61-6b7687db4723']


Let's check the datapacks names using the `/api/v1.0/datapacks` url

In [8]:
for dp in myself['datapacks']:
    print repovizz_client.get('/api/v1.0/datapacks/{}'.format(dp))['name']

MYO_various


Retrieve a Datapack
-------------------

In [11]:
datapack_id = myself['datapacks'][0] #datapack id to retrieve
datapack = repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))

This is the datapack structure. The first layer has metadata (owner, name, permissions...) and a member (`structure`) that is the datapack document.

In [12]:
datapack

{u'folder': None,
 u'group_edit_permissions': [],
 u'id': u'a4e6b685-afb8-4b40-9c61-6b7687db4723',
 u'name': u'MYO_various',
 u'owner': 1,
 u'structure': {u'children': [{u'class': u'data',
    u'link': u'repovizz:9ec009d2ea38b6d600f793424739dcdc75bf8a7c',
    u'meta': {u'mime': u'video/mp4; charset=binary'},
    u'name': u'Video',
    u'text': u'Handheld camera',
    u'type': u'e44ab9c4-a1b5-4474-b806-ece64555ef2b'},
   {u'class': u'data',
    u'link': u'repovizz:a910ba389ca2847b6bf548fadad68ccebeee4f54',
    u'meta': {u'mime': u'audio/x-wav; charset=binary'},
    u'name': u'Audio',
    u'text': u'Camera microphone',
    u'type': u'48e3b5d2-c34c-459c-9a2d-0756c20d80de'},
   {u'children': [{u'class': u'data',
      u'link': u'repovizz:0162912010aa5cad7f6d16bb487b4300c7e76942',
      u'name': u'EMG',
      u'text': u'8-channel EMG data from the MYO armband',
      u'type': u'0746bb48-9b92-4cd3-8d9e-3e1de13269eb'}],
    u'class': u'container',
    u'name': u'EMG',
    u'text': u''},
   {u

The leaves of the document have links to the data files. If the link starts with `repovizz:` it means that the file is hosted on repovizz2.

In [13]:
data_id = datapack['structure']['children'][2]['children'][0]['link']
print data_id

repovizz:0162912010aa5cad7f6d16bb487b4300c7e76942


To query the file, remove the `repovizz:` prefix and fetch it with the `/api/v1.0/datapacks/{id}/content/{data_id}` url.

In [14]:
#retrieve data
data_id = data_id.split(':')[1] #remove prefix
d = repovizz_client.get('/api/v1.0/datapacks/{}/content/{}'.format(datapack['id'],data_id),raw=True)

In [18]:
#this file is a json
print d.text[:200]

[
  [-1,1,0,-1,-1,-1,1,1  ],
  [-2,-3,-1,-3,0,-2,-3,-2  ],
  [-2,-3,-1,-3,0,-2,-3,-2  ],
  [-1,-1,-1,-1,1,0,-1,-2  ],
  [-1,-1,-1,0,0,0,-1,-1  ],
  [-1,-1,-1,0,0,0,-1,-1  ],
  [-1,-1,-1,0,0,0,-1,-1  ]


Continue with [Datapack Edition](Edition.ipynb)

Datapack Edition
================

Repovizz2 can be used to edit datapacks previously uploaded by yourself or other users.

Get a datapack, to be edited
----------------------------

In [ ]:
myself = repovizz_client.get("/api/v1.0/user")
datapack_id = myself['datapacks'][0] #datapack id to retrieve
datapack = repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))

In [ ]:
print datapack.keys()

Editing the Datapack
--------------------

To change a property, just post a dictionary with the modified elements

In [ ]:
repovizz_client.post(
    "/api/v1.0/datapacks/{}".format(datapack_id),
    json={
        'name': "New datapack Name",
    },
)

In [ ]:
repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))['name']

To modify the document (`structure`) you need to resubmit it all again

In [ ]:
document = datapack['structure']
print document['children'][0]['text']

In [ ]:
document['children'][0]['text'] = "Camera"
repovizz_client.post(
    "/api/v1.0/datapacks/{}".format(datapack_id),
    json={
        'structure': document,
    },
)

In [ ]:
repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))['structure']['children'][0]['text']

Adding a Data File
------------------

Let's modify the datapack by adding a new data file representing another sensor

In [ ]:
datapack = repovizz_client.get('/api/v1.0/datapacks/{}'.format(datapack_id))
document = datapack['structure']

In [ ]:
document['children'][2]['children'].append({
   u'class': u'data',
   u'link': u'emg 2.json',
   u'name': u'EMG 2',
   u'text': u'8-channel EMG data from the MYO armband, 2',
   u'type': u'0746bb48-9b92-4cd3-8d9e-3e1de13269eb'})

In [ ]:
document['children'][0]['text'] = "Camera"
repovizz_client.post(
    "/api/v1.0/datapacks/{}".format(datapack_id),
    json={
        'structure': document,
    },
)

Next, we upload the data file

In [ ]:
repovizz_client.get('/api/v1.0/datapacks/{}/content'.format(datapack_id)).keys()

In [ ]:
print repovizz_client.post(
    '/api/v1.0/datapacks/{}/content/{}'.format(datapack_id,'emg 2.json'),
    files={'file':open('MYO_example_datapack/emg 2.json')}
)